In [6]:
1 << 2

4

### 旅行商问题

In [81]:
from typing import List

class Solution:
    def shortestSuperstring(self, words: List[str]) -> str:
        def calcDis(w1, w2):
            # 用于表示在 w1的后面拼接 w2 的最小花费，最大就是w1的后端和w2的前端没有重叠
            # 那么代价就是 len(s2)
            n1, n2 = len(w1), len(w2)
            for i in range(min(n1, n2), -1, -1):
                if w1[-i:] == w2[:i]:
                    return n2 - i
            return n2
        
        def combine(w1, w2):
            # 将 w2 拼接在 w1 的后面，尽量重叠重合的部分
            n1, n2 = len(w1), len(w2)
            for i in range(min(n1, n2), -1, -1):
                if w1[-i:] == w2[:i]:
                    return w1 + w2[i:]
            return w1 + w2
        
        n = len(words)
        m = 1 << n
        
        # dis[i][j] 表示在i的后面拼接j，最小代价是多少
        dis = [[float('inf')] * n for _ in range(n)]
        for i in range(n):
            for j in range(n):
                if i == j:
                    dis[i][j] = 0
                else:
                    dis[i][j] = calcDis(words[i], words[j])
        
        # dp[status][last_word]: status是二进制的数字 0111 表示拼接 words[1]、word[2]、word[3]
        # 且最后一个拼接的单词是last word的最小代价是多少
        dp = [[float('inf')] * n for i in range(m)]
        # 如果只拼接一个单词，那么很显然，代价为这个单词本身的长度
        for i in range(n):
            dp[1 << i][i] = len(words[i])
            
        par = [[-1] * n for i in range(m)] # par[i][j] 表示在i的后面拼接j可以是代价最小化
        
        for s in range(m):
            for i in range(n): # 用于遍历，状态s使用了那些单词
                if (s & (1 << i)) == 0: # 如果当前单词没有使用到，直接跳过
                    continue
                pre_s = s - (1 << i) # pre_s 表示可以到达 s 的前一个状态
                if pre_s == 0: # 如果 s 只使用了一个单词的话，那么前驱状态 pre_s 必然是0
                    continue
                    
                for j in range(n): # 哪些状态可以达到前驱状态，这些拼接的单词，每一个都有可能是最后一个拼接的
                    if (pre_s & (1 << j)) == 0:
                        continue
                    # dis[j][i] 表示先拼接 j 在拼接 i 的代价，取一个最小值
                    if dp[pre_s][j] + dis[j][i] < dp[s][i]:
                        dp[s][i] = dp[pre_s][j] + dis[j][i]
                        par[s][i] = j # i前面是 j
        
        # 查找最后一个拼接的单词是什么
        end = None
        cost = float('inf')
        for i in range(n):
            if dp[-1][i] < cost:
                cost = dp[-1][i]
                end = i
        
        # 根据起点拼接
        node = end
        path = [node] # 最优路径
        status = m-1
        while par[status][node] != -1:
            pre_node = par[status][node] # 当前单词前面拼接什么是最优的
            path.append(pre_node) 
            status = status ^ (1 << node) # 或者 status - (1 << node)，表示去掉当前去过的地方
            node = pre_node
        
        # 因为我们是从后往前遍历的，所以要逆序
        res = words[path[-1]]
        for i in range(n-2, -1, -1):
            res = combine(res, words[path[i]])
        return res

In [82]:
solution = Solution()
solution.shortestSuperstring(["alex","loves","leetcode"])

'leetcodelovesalex'

In [ ]:
[[-1, -1, -1], 
 [-1, -1, -1], 
 [-1, -1, -1], 
 [1, 0, -1], 
 [-1, -1, -1], 
 [2, -1, 0], 
 [-1, 2, 1], 
 [1, 0, 0]] 
2 [0, 1, 2]
